In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        continue

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Dataset : [https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification/data]

## Importing Libraries

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd

## Preprocessing

In [3]:
data = {'Content': [], 'Label': []}
for dirname, _, filenames in os.walk('/kaggle/input/10dataset-text-document-classification'):
    for filename in filenames:
        # Read the content of the file
        file_path = os.path.join(dirname, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Use directory name as the label
        label = os.path.basename(dirname)

        # Append data to the dictionary
        data['Content'].append(content)
        data['Label'].append(label)

In [45]:
# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

                                               Content     Label
0    Archive-name: space/intro\nLast-modified: $Dat...     space
1    shag@aero.org (Rob Unverzagt) writes:\n>In art...     space
2    Does anyone know how to size cold gas roll con...     space
3    nanderso@Endor.sim.es.com (Norman Anderson) wr...     space
4    Here are some recent observations taken by the...     space
..                                                 ...       ...
995  US interest rate rise expected\n\nUS interest ...  business
996  House prices drop as sales slow\n\nHouse price...  business
997  UK economy facing 'major risks'\n\nThe UK manu...  business
998  Industrial revival hope for Japan\n\nJapanese ...  business
999  Peugeot deal boosts Mitsubishi\n\nStruggling J...  business

[1000 rows x 2 columns]


In [46]:
unique_labels = df['Label'].unique()
print("Unique Labels:")
print(unique_labels)

Unique Labels:
['space' 'politics' 'sport' 'technologie' 'historical' 'medical'
 'graphics' 'entertainment' 'food' 'business']


## Label Encoder

In [47]:
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

In [48]:
texts = df['Content'].tolist()
labels = df['Label'].tolist()

In [35]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Check if a GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

def predict_class(text, model, tokenizer, max_length=128, device='cpu'):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return preds.item()

## Fine Tuning Model Parameter

In [36]:
bert_model_name = 'bert-base-uncased'
num_classes = 10
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

# Define the model and move it to the device
model = BERTClassifier(bert_model_name, num_classes).to(device)

# Use stratify to ensure all labels get to train and test
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels)

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Train The Model

In [37]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    accuracy, report = evaluate(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

Epoch 1/4
Validation Accuracy: 0.7600
              precision    recall  f1-score   support

           0       0.34      0.90      0.49        20
           1       1.00      0.80      0.89        20
           2       1.00      0.95      0.97        20
           3       0.89      0.80      0.84        20
           4       0.94      0.75      0.83        20
           5       0.80      1.00      0.89        20
           6       1.00      0.25      0.40        20
           7       1.00      0.25      0.40        20
           8       1.00      1.00      1.00        20
           9       0.78      0.90      0.84        20

    accuracy                           0.76       200
   macro avg       0.87      0.76      0.76       200
weighted avg       0.87      0.76      0.76       200

Epoch 2/4
Validation Accuracy: 0.9350
              precision    recall  f1-score   support

           0       0.94      0.75      0.83        20
           1       1.00      1.00      1.00        20
  

## Making the Prediction

In [49]:
import torch

# Ensure that the model is on the GPU
model.to(device)

def predict_class(text, model, tokenizer, max_length=128, device='cpu'):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return preds.item()

# Test sentiment prediction
test_text = "This recipe is coming from my mother and originally use potato instead"
Class = predict_class(test_text, model, tokenizer, device=device)
print(f"Predicted sentiment: {Class}")

Predicted sentiment: 2


In [52]:
print(f"Predicted sentiment: {label_encoder.inverse_transform([Class])[0]}")

Predicted sentiment: food
